In [1]:
import unicodedata
import sys
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import translate, regexp_replace
import pyspark.sql.functions as f
from pyspark import SQLContext

### Global Variables ###
oij_file_name = "datos_delictivos_oij - Estadisticas-1.csv";
inec_file_name = "datos_inec.csv"

sc = pyspark.SparkContext()
sql = SQLContext(sc)

### ---------------- ###

def initialize_frames():
    oij_data_frame = (sql.read.format("com.databricks.spark.csv").option("header", "true").load(oij_file_name))
    inec_data_frame = (sql.read.format("com.databricks.spark.csv").option("header", "true").load(inec_file_name))
    return oij_data_frame,inec_data_frame

def remove_white_space(oij_data_frame,inec_data_frame):
    oij_data_frame = oij_data_frame.withColumn("Distrito",trim(oij_data_frame.Distrito))
    oij_data_frame = oij_data_frame.withColumn("Distrito",regexp_replace("Distrito"," ",""))

    inec_data_frame = inec_data_frame.withColumn("provincia_canton_distrito",trim(inec_data_frame.provincia_canton_distrito))
    inec_data_frame = inec_data_frame.withColumn("provincia_canton_distrito",regexp_replace("provincia_canton_distrito"," ",""))
    return oij_data_frame,inec_data_frame

def to_lowercase(oij_data_frame, inec_data_frame):
    oij_data_frame = oij_data_frame.withColumn("Distrito",f.lower(f.col("Distrito")))

    inec_data_frame = inec_data_frame.withColumn("provincia_canton_distrito",f.lower(f.col("provincia_canton_distrito")))
    return oij_data_frame,inec_data_frame

def show_data_frames(oij_data_frame,inec_data_frame):
    oij_data_frame.show()
    inec_data_frame.show()


def make_diff_list(oij_data_frame,inec_data_frame):
    oij_data_frame.createOrReplaceTempView("OIJ")
    inec_data_frame.createOrReplaceTempView("INEC")
    diff_list = sql.sql(
        "Select OIJ.Distrito from OIJ LEFT JOIN INEC ON OIJ.distrito = INEC.provincia_canton_distrito WHERE INEC.provincia_canton_distrito is NULL GROUP BY OIJ.Distrito");
    return diff_list

def show_diff_list(oij_data_frame,inec_data_frame):
    make_diff_list(oij_data_frame,inec_data_frame).show()

def show_how_many_are_diff(oij_data_frame,inec_data_frame):
    diff_list = make_diff_list(oij_data_frame,inec_data_frame)
    diff_list.createOrReplaceTempView("diff_table")
    diff_list = sql.sql("Select count(*) as cantidad from diff_table")
    diff_list.show();

def trans():
    matching_string = ""
    replace_string = ""

    for i in range(ord(" "), sys.maxunicode):
        name = unicodedata.name(chr(i), "")
        if "WITH" in name:
            try:
                base = unicodedata.lookup(name.split(" WITH")[0])
                matching_string += chr(i)
                replace_string += base
            except KeyError:
                pass

    return matching_string, replace_string

def clean(c):
    matching_string, replace_string = trans()
    return translate(
        regexp_replace(c, "\p{M}", ""),
        matching_string, replace_string
    ).alias(c)

def remove_accents(inec_data_frame):
    return inec_data_frame.select(clean("provincia_canton_distrito"))

In [2]:
show_data_frames(oij_data_frame,inec_data_frame)

NameError: name 'oij_data_frame' is not defined

In [3]:
a,b = initialize_frames()
show_data_frames(a,b)

+------+-----------+----------+-------------------+----------+--------------------+-------------+------------+--------------------+----------+------------+
|Delito|  SubDelito|     Fecha|            Victima|SubVictima|                Edad|       Genero|Nacionalidad|           Provincia|    Canton|    Distrito|
+------+-----------+----------+-------------------+----------+--------------------+-------------+------------+--------------------+----------+------------+
|ASALTO|ARMA BLANCA|2019-01-04|03:00:00 - 05:59:59|   PERSONA|    PEATON [PERSONA]|Mayor de edad|       MUJER|          COSTA RICA|PUNTARENAS|    GARABITO|
|ASALTO|ARMA BLANCA|2019-01-05|18:00:00 - 20:59:59|   PERSONA|   TAXISTA [PERSONA]|Mayor de edad|      HOMBRE|          COSTA RICA|   CARTAGO|     CARTAGO|
|ASALTO|ARMA BLANCA|2019-01-09|00:00:00 - 02:59:59|   PERSONA|    GUARDA [PERSONA]|Mayor de edad|      HOMBRE|          COSTA RICA|GUANACASTE|  SANTA CRUZ|
|ASALTO|ARMA BLANCA|2019-01-10|12:00:00 - 14:59:59|   PERSONA|  